In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
from torchvision import transforms
import time


# CPU 사용 (Jetson Nano 고려)
device = torch.device('cpu')

c:\Users\dlagu\anaconda3\envs\embed\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class TeacherVGG(nn.Module):
    def __init__(self, num_classes=10):
        super(TeacherVGG, self).__init__()
        
        self.features = nn.Sequential(
            # 첫 번째 블록
            nn.Conv2d(1, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # 두 번째 블록
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # 세 번째 블록
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((4, 4)),
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

classes = [
    'circle',        # 단순한 원
    'triangle',      # 삼각형
    'square',        # 사각형
    'donut',         # 초승달
    'house',      # 마름모
    'cloud',      # 팔각형
    'lightning',    # 번개
    'star',         # 별
    'diamond',      # 육각형
    'banana'          # 직선
]


# 모델 로드
def load_model(model_path):
    model = TeacherVGG(num_classes=len(classes))
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    return model

model = load_model('best_teacher_model.pth')

In [3]:
def preprocess_frame(frame):
    """비디오 프레임을 모델 입력으로 전처리"""
    # 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 이진화 (적응형 임계값 적용)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                 cv2.THRESH_BINARY_INV, 11, 2)
    
    # 노이즈 제거
    binary = cv2.medianBlur(binary, 3)
    
    # 윤곽선 검출 및 ROI 추출
    # OpenCV 버전 호환성을 위한 수정
    contours_output = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # OpenCV 버전에 따라 반환값이 다를 수 있으므로 적절히 처리
    contours = contours_output[-2] if len(contours_output) == 3 else contours_output[0]
    
    if contours:
        # 가장 큰 윤곽선 찾기
        main_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(main_contour)
        
        # 여백 추가
        padding = 20
        x = max(0, x - padding)
        y = max(0, y - padding)
        w = min(binary.shape[1] - x, w + 2*padding)
        h = min(binary.shape[0] - y, h + 2*padding)
        
        # ROI 추출
        roi = binary[y:y+h, x:x+w]
        
        # 정사각형으로 만들기
        size = max(w, h)
        square = np.zeros((size, size), dtype=np.uint8)
        start_x = (size - w) // 2
        start_y = (size - h) // 2
        square[start_y:start_y+h, start_x:start_x+w] = roi
        
        # 28x28로 리사이즈
        resized = cv2.resize(square, (28, 28))
    else:
        # 윤곽선이 없으면 빈 이미지
        resized = np.zeros((28, 28), dtype=np.uint8)
    
    # 정규화 및 텐서 변환
    normalized = resized.astype(np.float32) / 255.0
    tensor = torch.FloatTensor(normalized).unsqueeze(0).unsqueeze(0)
    
    # 정규화
    transform = transforms.Normalize((0.5,), (0.5,))
    tensor = transform(tensor)
    
    return tensor, binary

In [4]:
def run_inference(model, camera_id=0, duration=60):
    """실시간 카메라 추론"""
    # 카메라 설정
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print("카메라를 열 수 없습니다.")
        return
    
    # 성능 측정을 위한 변수들
    fps_list = []
    inference_times = []
    frame_count = 0
    start_time = time.time()
    last_time = start_time  # FPS 계산을 위한 이전 프레임 시간
    
    try:
        while (time.time() - start_time) < duration:
            ret, frame = cap.read()
            if not ret:
                break
            
            current_time = time.time()
            # 현재 FPS 계산 (이전 프레임과의 시간 차이 사용)
            fps = 1.0 / max(current_time - last_time, 0.001)  # 0으로 나누기 방지
            last_time = current_time
            
            # 추론 시작 시간
            inference_start = time.time()
            
            # 이미지 전처리
            tensor, binary = preprocess_frame(frame)
            
            # 추론
            with torch.no_grad():
                outputs = model(tensor)
                probs = F.softmax(outputs, dim=1)
                conf, pred = torch.max(probs, 1)
            
            # 추론 시간 계산
            inference_time = max((time.time() - inference_start) * 1000, 0.001)  # ms
            
            # 결과 시각화
            predicted_class = classes[pred.item()]
            confidence = conf.item() * 100
            
            # 결과 표시
            result_text = f"Prediction: {predicted_class}"
            conf_text = f"Confidence: {confidence:.1f}%"
            fps_text = f"FPS: {fps:.1f}"
            
            # 색상 설정 (높은 신뢰도: 녹색, 낮은 신뢰도: 빨간색)
            color = (0, 255, 0) if confidence > 50 else (0, 0, 255)
            
            cv2.putText(frame, result_text, (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            cv2.putText(frame, conf_text, (10, 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            cv2.putText(frame, fps_text, (10, 90),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            # Top-3 예측 표시
            probs_np = probs.squeeze().numpy()
            top3_idx = probs_np.argsort()[-3:][::-1]
            for i, idx in enumerate(top3_idx):
                text = f"#{i+1}: {classes[idx]} ({probs_np[idx]*100:.1f}%)"
                cv2.putText(frame, text, (10, 120 + i*30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            # 원본과 이진화 이미지 표시
            binary_display = cv2.resize(binary, (frame.shape[1]//2, frame.shape[0]))
            binary_display = cv2.cvtColor(binary_display, cv2.COLOR_GRAY2BGR)
            display = np.hstack([frame, binary_display])
            
            cv2.imshow('QuickDraw Inference', display)
            
            # 성능 메트릭 저장
            fps_list.append(fps)
            inference_times.append(inference_time)
            frame_count += 1
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except KeyboardInterrupt:
        print("Interrupted by user")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        
        # 성능 통계 출력
        if fps_list:  # 리스트가 비어있지 않은 경우에만 계산
            avg_fps = np.mean(fps_list)
            avg_inference = np.mean(inference_times)
            print(f"\nPerformance Statistics:")
            print(f"Average FPS: {avg_fps:.2f}")
            print(f"Average Inference Time: {avg_inference:.2f}ms")
            print(f"Total Frames: {frame_count}")

In [6]:
# 실시간 추론 실행
run_inference(model, camera_id=0, duration=60)

Interrupted by user

Performance Statistics:
Average FPS: 22.73
Average Inference Time: 4.90ms
Total Frames: 406


In [8]:
import torch
import pandas as pd

def compare_model_params(teacher_path, student_path):
    # 모델 클래스 정의 (이전에 사용한 것과 동일한 구조)
    class TeacherVGG(nn.Module):
        def __init__(self, num_classes=10):
            super(TeacherVGG, self).__init__()
            
            self.features = nn.Sequential(
                nn.Conv2d(1, 64, 3, padding=1),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.Conv2d(64, 64, 3, padding=1),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, 2),
                
                nn.Conv2d(64, 128, 3, padding=1),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 3, padding=1),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, 2),
                
                nn.Conv2d(128, 256, 3, padding=1),
                nn.BatchNorm2d(256),
                nn.ReLU(inplace=True),
                nn.Conv2d(256, 256, 3, padding=1),
                nn.BatchNorm2d(256),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, 2),
            )
            
            self.classifier = nn.Sequential(
                nn.AdaptiveAvgPool2d((4, 4)),
                nn.Flatten(),
                nn.Linear(256 * 4 * 4, 512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(512, num_classes)
            )
    
    class CompactStudentNet(nn.Module):
        def __init__(self, num_classes=10):
            super(CompactStudentNet, self).__init__()
            
            self.features = nn.Sequential(
                nn.Conv2d(1, 16, 3, padding=1),
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, 2),
                
                nn.Conv2d(16, 32, 3, padding=1),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, 2),
                
                nn.Conv2d(32, 32, 3, padding=1, groups=32),
                nn.Conv2d(32, 64, 1),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, 2),
            )
            
            self.classifier = nn.Sequential(
                nn.AdaptiveAvgPool2d((2, 2)),
                nn.Flatten(),
                nn.Linear(64 * 2 * 2, num_classes)
            )
    
    # 모델 인스턴스 생성
    teacher_model = TeacherVGG()
    student_model = CompactStudentNet()
    
    # 체크포인트 로드
    teacher_ckpt = torch.load(teacher_path, map_location='cpu')
    student_ckpt = torch.load(student_path, map_location='cpu')
    
    # 모델에 가중치 로드
    teacher_model.load_state_dict(teacher_ckpt['model_state_dict'])
    student_model.load_state_dict(student_ckpt['model_state_dict'])
    
    # Teacher 모델 정보
    print("=== Teacher Model ===")
    teacher_total = sum(p.numel() for p in teacher_model.parameters())
    teacher_train = sum(p.numel() for p in teacher_model.parameters() if p.requires_grad)
    print(f"Total parameters: {teacher_total:,}")
    print(f"Trainable parameters: {teacher_train:,}")
    print(f"Best accuracy: {teacher_ckpt['accuracy']:.2f}%")
    
    # Student 모델 정보
    print("\n=== Student Model ===")
    student_total = sum(p.numel() for p in student_model.parameters())
    student_train = sum(p.numel() for p in student_model.parameters() if p.requires_grad)
    print(f"Total parameters: {student_total:,}")
    print(f"Trainable parameters: {student_train:,}")
    print(f"Best accuracy: {student_ckpt['accuracy']:.2f}%")
    
    # 모델 크기 비교
    print("\n=== Model Size Comparison ===")
    reduction = (1 - student_total/teacher_total) * 100
    print(f"Size reduction: {reduction:.1f}%")

    return {
        'teacher': {
            'total_params': teacher_total,
            'trainable_params': teacher_train,
            'accuracy': teacher_ckpt['accuracy']
        },
        'student': {
            'total_params': student_total,
            'trainable_params': student_train,
            'accuracy': student_ckpt['accuracy']
        }
    }

# 사용 예시:
teacher_path = 'best_teacher_model.pth'
student_path = 'best_student_model.pth'
stats = compare_model_params(teacher_path, student_path)

# 결과를 데이터프레임으로 정리
df = pd.DataFrame({
    'Model': ['Teacher', 'Student'],
    'Total Parameters': [stats['teacher']['total_params'], stats['student']['total_params']],
    'Trainable Parameters': [stats['teacher']['trainable_params'], stats['student']['trainable_params']],
    'Accuracy': [stats['teacher']['accuracy'], stats['student']['accuracy']]
})
print("\n=== Summary ===")
print(df)

=== Teacher Model ===
Total parameters: 3,248,842
Trainable parameters: 3,248,842
Best accuracy: 96.53%

=== Student Model ===
Total parameters: 10,026
Trainable parameters: 10,026
Best accuracy: 95.44%

=== Model Size Comparison ===
Size reduction: 99.7%

=== Summary ===
     Model  Total Parameters  Trainable Parameters  Accuracy
0  Teacher           3248842               3248842     96.53
1  Student             10026                 10026     95.44
